In [7]:
import os
from dotenv import load_dotenv
load_dotenv()



True

In [22]:
### Basic Prompt Summarization

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [31]:
speech = """
My dear countrymen, this nation has neither been built by political leaders nor by rulers nor by governments. This nation has been built by our farmers, our workers, our mothers and sisters, our youth. The country has reached here today because of generation to generation rigours undertaken by our sages, our saints, our maestros, our teachers, our scientists and social workers. These great people and these great generations, who had worked for the country throughout their lives, deserve our deepest respect. This is the beauty of India's Constitution, this is its capability which has made it possible that today a boy from small town, a poor family has got the opportunity to pay homage to the tri-colour of India at the ramparts of 
Lal Quila (Red Fort). This is the strength of India's democracy. This is an invaluable legacy which we have inherited from our architects of the constitution. I pay my respects to those architects of the constitution of India today.

Brothers and sisters, today if we have reached here after independence, it is because of the contribution of all the Prime Ministers, all the governments and even the governments of all the States. I want to express my feelings of respect and gratitude to all those previous governments and ex-Prime Ministers who have endeavoured to take our present day India to such heights and who have added to the country's glory.

This country has been built on such foundation of ancient cultural heritage, where we were told of only one
mantra during Vedic period, which is indicative of our work culture, which we have learnt, we have memorized – “
Sangachchhdhvam Samvadadhvam sam wo manansi jaanataam.” We walk together, we move together, we think together, we resolve together and together we take this country forward. Having imbibed this basic mantra, 125 crores of countrymen have taken the nation forward. Yesterday only the first Parliamentary Session of the new Government had concluded. Today, I can proudly say that the Session of Parliament reflects our thinking and it is a reflection of our intentions. We are not for moving forward on the basis of majority, we are not interested to move forward by virtue of majority. We want to move ahead on the basis of strong consensus. 
"Sangachhadhwam" and, therefore, the nation has witnessed the entire Session of Parliament. Having taken all the Parties and Opposition along while working shoulder to shoulder, we achieved an unprecedented success and the credit for this does not go to the Prime Minister alone, the credit does not go to the people sitting in the Government, the credit for this goes to the Opposition also, the credit goes to all the leaders of Opposition too and also all the Members from Opposition. From the ramparts of Red Fort, quite proudly I salute all the Members of Parliament, I also salute all the Political Parties and by virtue of their strong support, we could take some important decisions intended to take the nation forward and yesterday the Session of Parliament had concluded.
"""


In [36]:
chat_messages = [
    SystemMessage(content='You are an expert assitant with expertise in summarizing speeches'),
    HumanMessage(content=f'Please provide us short and concise summary of the following speech:\n TEXT: {speech}'),
]

llm=ChatOpenAI(model_name="gpt-3.5-turbo")

In [33]:
llm.get_num_tokens(speech)


615

In [37]:
llm(chat_messages)

AIMessage(content="The speech emphasizes that the nation's progress is a result of the hard work and dedication of various individuals such as farmers, workers, mothers, youth, sages, saints, teachers, scientists, and social workers. It acknowledges the contribution of past governments, Prime Ministers, and the importance of consensus in moving the country forward. The speaker highlights the importance of unity, cultural heritage, and collaboration between the government and opposition for the nation's development. The emphasis on working together for the country's progress is highlighted as a key principle, and gratitude is expressed towards all those who have contributed to the country's growth.")

In [38]:
print(llm(chat_messages).content)

The speech highlights the role of the people, including farmers, workers, mothers, youth, sages, saints, teachers, scientists, and social workers, in building the nation. It acknowledges the contributions of past governments, Prime Ministers, and the strength of India's democracy. The importance of unity and consensus in moving the country forward is emphasized, with a focus on working together regardless of political differences. The speaker expresses gratitude to all Members of Parliament and political parties for their support in achieving success during the recent Parliamentary Session.


<p style="font-size: 200%;">Prompt Templates Text Summarization</p>

In [39]:
#before that it was directlt using human message etc.
from langchain.chains import LLMChain
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore .document import Document

In [40]:
#we will create custom prompts and givt to llm
generic_template = '''
Write a summary of the following speech:
Speech: `{speech}`
Translate the precise sumary to {language}.
'''

prompt = PromptTemplate(
    input_variables=['speech', 'language'],
    template=generic_template
)

In [42]:
complete_prompt = prompt.format(speech=speech, language='Hindi')
complete_prompt

'\nWrite a summary of the following speech:\nSpeech: `\nMy dear countrymen, this nation has neither been built by political leaders nor by rulers nor by governments. This nation has been built by our farmers, our workers, our mothers and sisters, our youth. The country has reached here today because of generation to generation rigours undertaken by our sages, our saints, our maestros, our teachers, our scientists and social workers. These great people and these great generations, who had worked for the country throughout their lives, deserve our deepest respect. This is the beauty of India\'s Constitution, this is its capability which has made it possible that today a boy from small town, a poor family has got the opportunity to pay homage to the tri-colour of India at the ramparts of \nLal Quila (Red Fort). This is the strength of India\'s democracy. This is an invaluable legacy which we have inherited from our architects of the constitution. I pay my respects to those architects of t

In [43]:
llm.get_num_tokens(complete_prompt)

635

In [47]:
llm_chain = LLMChain(llm=llm,prompt=prompt)
summary = llm_chain.run({'speech': speech, 'language': 'Hindi'})

In [48]:
summary

'मेरे प्यारे देशवासियों, यह राष्ट्र न किसी राजनेताओं ने बनाया है, न किसी शासकों ने, न किसी सरकारों ने। यह राष्ट्र हमारे किसानों, हमारे मज़दूरों, हमारी माएं और बहनों, हमारे युवाओं ने बनाया है। देश आज यहाँ पहुंचा है क्योंकि हमारे ऋषियों, महात्माओं, माहेरों, शिक्षकों, वैज्ञानिकों और सामाजिक कार्यकर्ताओं ने पीढ़ी से पीढ़ी कठिन परिश्रम किया है। इन महान लोगों और इन महान पीढ़ियों का हमें सबसे गहरा सम्मान देना चाहिए। भारत की संविधान की यह सुंदरता है, यह उसकी क्षमता है जिसने यह संभव बनाया है कि आज एक छोटे शहर के लड़के, एक गरीब परिवार से, ल'

<p style="font-size: 200%;">StuffDocumentChain Text Summarization</p>
If more than 4096 tokens, use stuffdocumentation

In [50]:
from PyPDF2 import PdfReader
#If I have content in a document, it iwll push to llm model  and get response... Get summarized version directly

In [51]:
pdfreader = PdfReader("apjspeech.pdf")

In [52]:
#enumerate through pdf all age and extract text
from typing_extensions import Concatenate
text=''
for i,page in enumerate(pdfreader.pages):
    content = page.extract_text()
    text+=content

In [53]:
text

'A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi and at every state and union territor y as well as through my \nonline interactions, I have many unique experiences to share with you, which signify the \nfollowing important messages: \n \n1. Accelerate development : Aspiration of th

In [55]:
#If I am using stuffchain, convert this to documnents(like above one has text worth 2 pages) because in this tech thats what llm model expects
from langchain.docstore .document import Document
docs = [Document(page_content=text)]
docs #all text in 1 doc

[Document(page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi and at every state and union territor y as well as through my \nonline interactions, I have many unique experiences to share with you, which signify the \nfollowing important messages: \n \n1. Accelerate develop

In [67]:
from langchain.chains.summarize import load_summarize_chain #-> chain_type -> stuff or map reduce
template='''
Write a concise and short summary of the following speech
Speech: `{text}`
'''
prompt = PromptTemplate(
    input_variables=['text'],
    template=template
)

In [59]:
chain=load_summarize_chain(
    llm,
    chain_type='stuff',
    prompt=prompt,
    verbose=True
)
output_summary = chain.run(docs)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Write a concise and short summary of the following speech
Speech: `A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online in

In [60]:
output_summary

'In his departing speech, A P J Abdul Kalam reflects on his five years in Rashtrapati Bhavan and highlights key messages such as accelerating development, empowering villages, mobilizing rural core competence, and overcoming challenges through partnership. He shares inspiring stories of individuals who have shown courage and resilience in the face of calamities. Kalam emphasizes the importance of youth empowerment and values-based education in achieving a developed India by 2020. He calls for unity, transparency, and eradication of poverty and discrimination in order to create a prosperous, peaceful, and happy nation. Kalam concludes by expressing gratitude and sharing his mission to connect the hearts and minds of all Indians in the journey towards a developed nation.'

We had documents which had limited data, we were able to document the entire text and get a quick reeponse
But when pdf of huge size, MapReduce documents


<p style="font-size: 200%;">Summzarizing Documents Using map Reduce</p>

In [61]:
#huge pdf divided into chunks, give each chunk to llm, each chunk has summary, then give llm model all the summaries and get final summary
from langchain import PromptTemplate
from langchain.chat_models  import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [68]:
pdfreader = PdfReader("apjspeech.pdf")
text=''
for i,page in enumerate(pdfreader.pages):
    content = page.extract_text()
    text+=content

In [69]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [70]:
llm.get_num_tokens(text)

4001

In [71]:
#split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)
chunks = text_splitter.create_documents([text])

In [73]:
len(chunks)

2

In [77]:
chain = load_summarize_chain(
    llm, 
    chain_type='map_reduce', 
    verbose=False)
summary1 = chain.run(chunks)

In [78]:
summary1

"In his departing speech, A P J Abdul Kalam reflects on his five years at Rashtrapati Bhavan and emphasizes the need to accelerate development based on the aspirations of the youth, empower villages for self-sufficiency, prioritize agriculture, and overcome challenges through partnership and courage. He shares inspiring stories of resilience and determination, and envisions a developed India by 2020 with reduced rural-urban divide, accessible energy and water, inclusive education, universal healthcare, transparent governance, and eradication of poverty and crime. Dr. Kalam calls for unity, hard work, and determination to achieve these goals, expressing confidence in India's potential for success."

<p style="font-size: 200%;">Map Reduce with Custom Prompts</p>

In [79]:
#We converted docs to chunks and we give chunks to llm, while giving, we can give custom prompts, for chunks and entore summary as well
chunks_prompt="""
    Please summarzie the below speech:
    Speech:     `{text}`
    Summary:
"""
map_prompt_template = PromptTemplate(
    input_variables=['text'],
    template=chunks_prompt)


In [80]:
final_combine_prompt ='''
Please provide  final summary of the entire speech with these important points.
Add a generic Motivational title,
Start the precise summary with an introduction and provide the 
summary in number points for the speech.
Speech: `{text}`
'''

final_combine_prompt_template = PromptTemplate(
    input_variables=['text'],
    template=final_combine_prompt)

In [81]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=final_combine_prompt_template,
    verbose=False)

output =  summary_chain.run(chunks)

In [82]:
output

'Title: "Inspiring Unity and Progress: A Vision for a Developed India"\n\n1. Dr. A. P. J. Abdul Kalam reflects on his five years at Rashtrapati Bhavan, emphasizing the importance of accelerating development and empowering villages.\n2. He shares inspiring stories of resilience and courage, highlighting the significance of positive attitude and perseverance in overcoming challenges.\n3. Kalam stresses the importance of agriculture as the backbone of the nation and the need to empower farmers for economic growth.\n4. He discusses his proposal for a Pan African e-Network to connect African countries and improve access to education and healthcare.\n5. Dr. Kalam praises the dedication of India\'s defense forces and emphasizes the empowerment of youth for societal transformation.\n6. He outlines his vision for a developed India by 2020, emphasizing unity, equality, and progress in various sectors.\n7. Dr. Kalam expresses gratitude for the support of citizens and reaffirms his commitment to t

<p style="font-size: 200%;">Refine Chain for Summarization</p>
We give chunk 1, get summary, then ww combine chaunk 1 and chunk2, get a summary, 3rd, chunk1,2,3 combine and get summarized version
Used for huge document

In [84]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    verbose=True
)
output_summary = chain.run(chunks)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have man

In [85]:
output_summary

"In his departing speech, A P J Abdul Kalam reflects on his five years as President of India and highlights important messages he has gathered during his interactions with various sectors of society. He emphasizes the need to accelerate development based on the aspirations of the youth, empower villages for self-sufficiency, mobilize rural core competence for competitiveness, prioritize agriculture as the backbone of the nation, and overcome challenges through partnership and courage. He shares inspiring stories of individuals and communities who have shown resilience and determination in the face of disasters and challenges. Kalam calls for a united effort towards a Developed India by 2020, focusing on the empowerment of youth and rural communities. Additionally, he discusses his efforts in promoting connectivity for societal transformation, defending the nation's pride through interactions with the armed forces, and supporting youth movements for a Developed India by 2020. Kalam conc